In [612]:
import requests
import json
import pandas as pd
import time

# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running flask from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

# Running with gunicorn (heroku, digitalocean, etc):
# Procfile >
# web: gunicorn wsgi --log-file -
# wsgi.py >
# from api import app as application
#
#if __name__ == "__main__":
#    application.run()

# https://devcenter.heroku.com/articles/getting-started-with-python
# https://www.digitalocean.com/community/tutorials/how-to-serve-flask-applications-with-gunicorn-and-nginx-on-ubuntu-14-04

In [613]:
# Set up heroku database > 
# heroku addons:create heroku-postgresql:hobby-dev
# Postgres db now up
# Look at config vars (should have DATABASE_URL) >
# heroku config
# Connect to remote psql > 
# heroku pg:psql
#


In [618]:
#  baseurl = 'https://vast-depths-89393.herokuapp.com/'
# baseurl = 'https://glacial-hollows-90395.herokuapp.com/'
#baseurl = 'http://127.0.0.1:5000/'
baseurl = 'http://127.0.0.1:7000/'

In [619]:
url = baseurl +'createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [620]:
response.json()

{'signingKey_hex': '17faa0ebe336a38575c3c127c00279eb38b8f01410c6183f1eac87dca3c85faf',
 'traderId': '5',
 'verifyKey_hex': '06d532a9d582a425619f6aabafa54b20b1883a6daefd646aeab993a6a656df57'}

In [621]:
url = baseurl + 'createMarket'

mkRtId = 6
mkId=1
# copy from response
sk = response.json()['signingKey_hex']
vk = response.json()['verifyKey_hex']
tId = response.json()['traderId']

In [622]:
content_makemarket = {"signingKey_hex": sk,
                    "traderId": tId, 
                    "verifyKey_hex": vk,
                    "marketRootId": mkRtId, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)

In [623]:
pd.DataFrame(response.json(), index=[0])

,checks,marketBranchId,marketMax,marketMin,marketRootId,traderId
0,False,1,1,0,6,5


In [624]:
# Create a trade
url = baseurl+'createTrade'

content_maketrade = {"signingKey_hex": sk,
                     "traderId": tId,
                     "verifyKey_hex": vk,
                     "marketId": mkId,
                     "price": 0.55,
                     "quantity":1}

In [625]:
# Post trade
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [626]:
response.json()

{'checks': 'True',
 'marketId': 1,
 'price': 0.55,
 'quantity': 1,
 'traderId': '5'}

In [594]:
# View json response
pd.DataFrame(response.json(), index=[0])


,checks,marketId,price,quantity,traderId
0,True,1,0.55,1,3


In [595]:
# Create a matched trade
url = baseurl+'createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [596]:
response.json()

{'checks': 'True',
 'marketId': 1,
 'price': 0.55,
 'quantity': -1,
 'traderId': '3'}

In [597]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(5):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 0.7508418560028076 seconds.
matched trade: 1 in 0.6976377964019775 seconds.
matched trade: 2 in 0.6942172050476074 seconds.
matched trade: 3 in 0.6900448799133301 seconds.
matched trade: 4 in 0.7006101608276367 seconds.


In [627]:
url = baseurl+'createMarket'

for i in range(4):
    t = time.time()

    content_makemarket = {"signingKey_hex": sk,
      "traderId": int(tId), "verifyKey_hex": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

failed to create market 1
failed to create market 2
failed to create market 3
failed to create market 4


In [628]:
tId

'5'

In [629]:
response.json()

{'checks': 'False',
 'marketBranchId': 1,
 'marketMax': 0.7,
 'marketMin': 0,
 'marketRootId': 4,
 'traderId': 5}

In [601]:
url = baseurl+'viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)



In [602]:
a = pd.read_json(response.json())
a

,marketId,marketRootId,marketBranchId,marketMin,marketMax
0,1,1,1,0.0,1.0
1,2,1,2,0.1,0.9
2,3,2,1,0.0,1.0
3,4,6,1,0.0,1.0
4,5,3,1,0.0,0.7
5,6,4,1,0.0,0.7


In [603]:
url = baseurl+'viewOrderBook'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [604]:
a = pd.read_json(response.json())
a

,marketId,price,quantity,traderId,iMatched
0,3,0.50,-1.00,2,False
1,1,0.55,0.01,4,False
10,3,0.50,1.00,1,True
11,3,0.50,-1.00,2,True
12,1,0.55,1.00,3,True
13,1,0.55,-1.00,3,True
14,1,0.55,1.00,3,True
15,1,0.55,-1.00,3,True
16,3,0.50,1.00,2,True
17,3,0.50,-1.00,1,True


In [605]:
# Create many unmatched trades
url = baseurl+'createTrade'

for i in range(5):
    t = time.time()
    content_maketrade['quantity'] = -1
    content_maketrade['price'] = 0.43
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('unmatched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


unmatched trade: 0 in 1.3190641403198242 seconds.
unmatched trade: 1 in 1.2798469066619873 seconds.
unmatched trade: 2 in 1.2401010990142822 seconds.
unmatched trade: 3 in 1.24934983253479 seconds.
unmatched trade: 4 in 1.2910048961639404 seconds.


In [606]:
response.json()

{'checks': 'True',
 'marketId': 1,
 'price': 0.43,
 'quantity': -1,
 'traderId': '3'}

In [607]:
url = baseurl+'viewOpenTrades'

content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [608]:
pd.read_json(response.json())

,marketId,price,quantity,traderId
0,1,0.43,-5.00,3
1,1,0.55,2.00,3
2,1,0.55,0.01,4
3,3,0.50,-1.00,2


In [609]:
url = baseurl+'viewMatchedTrades'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [610]:
pd.read_json(response.json())

,marketId,price,quantity,traderId
0,1,0.55,-2,3
1,1,0.55,2,4
2,3,0.50,0,1
3,3,0.50,0,2


In [611]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(5):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 2.527970790863037 seconds.
matched trade: 1 in 2.525451183319092 seconds.
matched trade: 2 in 2.4710869789123535 seconds.
matched trade: 3 in 2.485991954803467 seconds.
matched trade: 4 in 2.4780328273773193 seconds.


In [631]:
import datetime


In [638]:
datetime.date('today')

TypeError: an integer is required (got type str)